##**Trabalho Prático 1**



## Bárbara  Faria (A85774) , Bruna Araújo (A84408) e Tiago Lima (A85126)



## 2.Sudoku

Da definição do jogo “Sudoku” generalizado para a dimensão $N$; o problema tradicional corresponde ao caso $N=3$. O objetivo do Sudoku é preencher uma grelha de $\,N^2\times N^2\,$ com inteiros positivos no intervalo $\,1$ até $\,N^2\,$, satisfazendo as seguintes regras:
     

  1. Cada inteiro no intervalo $\,1$ até $\,N^2\,$ ocorre  só uma vez em cada coluna, linha e secção $\,N\times N\,$.

  2. No início do jogo uma fração $\,0\leq \alpha< 1\,$$ das $$\,N^4\,$ casas da grelha são preenchidas de forma consistente com a regra anterior.

Neste segundo exercício do Trabalho Prático 1 foi nos proposto:

  1. Construir um programa para inicializar a grelha a partir dos parâmetros $N$ e $\alpha$

  2. Construir soluções do problema para  as combinações de parâmetros $N\in\{3,4,5,6\}$  e $\,\alpha \in \{\,0.0\,,\,0.2\,,\,0.4\,,\,0.6\,\}$ . Que conclusões pode tirar da complexidade computacional destas soluções.


In [ ]:
!pip install ortools

###Análise do Problema

Existem $l$ linhas, identificadas por um índice $l \in [0..L\!-\!1]$ e $c$ colunas também identificadas por um índice $c \in [0..C\!-\!1]$ que fazem parte um Tabuleiro de Sudoku.

Vamos usar uma família $S_{l,c,n}$ de variáveis binárias  , com a seguinte semântica

$$S_{l,c,n} == 1  \quad \mbox{se e só se} \quad \mbox{o inteiro  $n$ está localizado na linha $l$ e na coluna $c$ } $$


In [ ]:
from ortools.linear_solver import pywraplp
import random

sudoku = pywraplp.Solver.CreateSolver('SCIP')

N = 3
T = N * N
A = T * T
ALPHA=0.4

Destaca-se ainda o seguinte:

**Limitações:** (que impõem limites máximos à alocação)
1. Cada inteiro ($0$..$T-1$) ocorre só uma vez em cada coluna.
2. Cada inteiro ($0$..$T-1$) ocorre só uma vez em cada linha.
3. Cada inteiro ($0$..$T-1$) ocorre só uma vez em cada seccão (grelha N*N)





Declaração das matrizes de alocação:

In [ ]:
s = {}
for l in range(T):
  s[l] = {}
  for c in range(T):
    s[l][c] = {}
    for n in range(T):
      s[l][c][n] = sudoku.BoolVar('s[%i][%i][%i]' % (l,c,n))

def S(l,c,n):
  return s[l][c][n]    

Passamos agora á modelação das restrições.


###Limitações:

1. Cada inteiro (1..T) ocorre só uma vez em cada coluna.

  Vamos percorrer todas as colunas e todos os inteiros e fazemos o somatório de todas as linhas e esse somatório tem de ser igual a 1, o que faz que em cada inteiro ocorra só uma vez por cada coluna.

$$\forall{c<T} \cdot \forall{n<T} \cdot \quad \sum_{l<T} s_{l,c,n} == 1$$

In [ ]:

for c in range(T):
  for n in range(T):
    sudoku.Add(sum([S(l,c,n) for l in range(T)]) == 1)

  2. Cada inteiro ($1$..$T$) ocorre só uma vez em cada linha

    Vamos percorrer todas as linhas e todos os inteiros e fazemos o somatório de todas as colunas e esse somatório tem de ser igual a 1, o que faz que em cada inteiro ocorra só uma vez por cada linha.

$$\forall{l<T} \cdot \forall{n<T} \cdot \quad \sum_{c<T} s_{l,c,n} == 1$$


In [ ]:
for l in range(T):
  for n in range(T):
    sudoku.Add(sum([S(l,c,n) for c in range(T)]) == 1)

  3. Cada inteiro ($0$..$T-1$) ocorre só uma vez em cada linha

    Vamos percorrer as linhas e as colunas de uma seccão (subgrelha N*N) e se o somatório for igual a 1, então cada inteiro ocorre apenas uma vez na mesma. A progressão das variáveis p e q na fórmula seguinte é de N em N.

$$\forall{n<T} \cdot \forall{p<T-1} \cdot \forall{q<T-1} \cdot \quad \sum_{p\leq l<p+N, \quad q\leq c<q+N} s_{l,c,n} == 1$$

In [ ]:
for n in range(T):
  for p in range(0, T-1, N):
    for q in range(0, T-1, N):
      sudoku.Add(sum([S(l,c,n) for l in range(p, p+N) for c in range(q, q+N)]) == 1)

###Exemplos

1. Cria um tabuleiro completo de raíz.

In [ ]:
status=sudoku.Solve()
if status==pywraplp.Solver.OPTIMAL:
  #imprime tabuleiro de sudoku
  for c in range(T):
    for l in range(T):
      for n in range(T):
        if round(S(l,c,n).solution_value())==1:
          print(" ",n," ",end='')
    print("\n")
else:
  print("Sem solução!")

  0    1    3    7    2    4    8    5    6  

  5    8    1    7    6    0    2    4    3  

  2    4    6    3    0    8    1    5    7  

  6    8    1    5    3    0    2    7    4  

  3    0    2    4    7    1    6    5    8  

  5    0    2    4    8    7    1    3    6  

  7    0    6    5    4    8    1    3    2  

  5    6    3    8    1    2    4    7    0  

  2    4    1    0    3    8    5    7    6  



2. Devolve um tabuleiro com uma percentagem (ALPHA) de posições preenchidas.

In [ ]:

comb = []
for c in range(T):
  for l in range(T):
    comb.append((l,c))

rand = random.choices(comb, k = round(ALPHA*A))

status=sudoku.Solve()
if status==pywraplp.Solver.OPTIMAL:
  #imprime tabuleiro de sudoku
  for c in range(T):
    for l in range(T):
      for n in range(T):
        if round(S(l,c,n).solution_value())==1:
          if (l,c) in rand:
            print(" ",n," ",end='')
          else:
            print(" ","_"," ",end='')
    print("\n")
else:
  print("Sem solução!")

  0    1    _    _    _    _    _    5    _  

  _    _    _    _    6    0    2    4    _  

  _    _    _    3    _    8    1    5    7  

  _    _    _    _    3    0    2    7    _  

  _    _    2    4    7    1    6    5    8  

  5    _    _    _    _    _    1    3    _  

  _    0    _    _    _    _    1    3    _  

  _    6    _    _    1    _    _    _    _  

  _    _    1    _    _    _    _    _    _  

